In [1]:
# reddit crawler
import pandas as pd
import praw
from LSTM_predict import *

your_client_id='ElyKc6o3du1IMb5LP2HYjg'
your_client_secret='YimfVn3bTLyVFu_XkJuLxrNkR3vHAQ'
your_user_name='guntsv'

reddit = praw.Reddit(client_id=your_client_id,
                     client_secret=your_client_secret,
                     user_agent=your_user_name,
                     check_for_async=False)

subreddit = reddit.subreddit('bitcoin')
topics = [*subreddit.top(limit=500)] # top posts all time
print(len(topics))

title = [n.title for n in topics]
df_topics = pd.DataFrame({"title": title})

df_result = pd.DataFrame(prediction(df_topics['title']))
df_result = df_result.rename(columns={0: 'Title', 1: 'Rating'})
df_result.head()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


500


,Title,Rating
0,"It's official! 1 Bitcoin = $10,000 USD",3
1,The last 3 months in 47 seconds.,1
2,Tesla buys $1.5b in Bitcoin and is looking to ...,3
3,It's over 9000!!!,3
4,Everyone who's trading BTC right now,2


In [2]:
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
nlp = spacy.load('en_core_web_md')
# ruler = nlp.add_pipe("entity_ruler")
# ruler.from_disk(skill_path)
# nlp.pipe_names

In [3]:
def preprocessing(sentence):

    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []

    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
                token.pos_ != 'SYM':
            cleaned_tokens.append(token.lemma_.lower().strip())

    return " ".join(cleaned_tokens)

In [4]:
df_result['clean'] = df_result['Title'].apply(preprocessing)
df_result['clean'].head()

0                   it official 1 bitcoin = 10,000 usd
1                                the 3 month 47 second
2    tesla buy 1.5b bitcoin look accept crypto form...
3                                              it 9000
4                             everyone trade btc right
Name: clean, dtype: object

In [11]:
negative = df_result['Rating'].isin([0,1])
postive = df_result['Rating'].isin([3,4])
#I considerly remove 2 as normal

In [16]:
df_pos = df_result[postive]
df_neg = df_result[negative]
df_pos.shape,df_neg.shape

((89, 3), (166, 3))

In [23]:
#Positive
#1. tokenize
#data cleaned
corpus = df_pos['clean']
#data tokenized
corpus_tokenized = [sent.split(" ") for sent in corpus]
#2. numericalize (vocab)
#2.1 get all the unique words
#we want to flatten unit (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs_pos = list(flatten(corpus_tokenized))

voc_size = len(vocabs_pos)
voc_size

893

In [24]:
word_freq = Counter(vocabs_pos)
common_words_pos = word_freq.most_common(10)
print (common_words_pos)

[('bitcoin', 50), ('i', 28), ('today', 13), ('btc', 11), ('buy', 9), ('year', 9), ('the', 9), ('good', 8), ('time', 8), ('new', 7)]


In [25]:
#Negative
#1. tokenize
#data cleaned
corpus = df_neg['clean']
#data tokenized
corpus_tokenized = [sent.split(" ") for sent in corpus]
#2. numericalize (vocab)
#2.1 get all the unique words
#we want to flatten unit (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs_neg = list(flatten(corpus_tokenized))

voc_size = len(vocabs_neg)
voc_size

1870

In [26]:
word_freq = Counter(vocabs_neg)
common_words_neg = word_freq.most_common(10)
print (common_words_neg)

[('bitcoin', 106), ('i', 43), ('the', 21), ('buy', 21), ('like', 19), ('money', 16), ('year', 13), ('time', 12), ('go', 11), ('btc', 10)]
